In [1]:
%conda install -q -y colorcet

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.transform import transform
import colorcet as cc

#Bokeh config
output_notebook()
tools = "hover, box_zoom, undo, crosshair, wheel_zoom, pan, zoom_in, zoom_out"

Loading BokehJS ...

In [10]:
# Constants
SIZE_X = 100
SIZE_Y = 100

ORIGIN_X = np.floor(SIZE_Y/2.0)
ORIGIN_Y = np.floor(SIZE_Y/2.0)

#ORIGIN_X = 0
#ORIGIN_Y = 0

SCALE = 0.1 # Value in meter for each pixel

TX_POWER = 0 #dBm
REFERENCE_LOSS_1M = 0 #dBm
PATH_LOSS_EXP = 2

#Plot constants
ALPHA=1.0

In [11]:
# Calculate distance between coordinates using pythagoras theorem
def distance2d(src_x, src_y, dst_x, dst_y):
    dx = (dst_x-src_x)*SCALE
    dy = (dst_y-src_y)*SCALE
    return np.sqrt((dx**2)+(dy**2))

In [12]:
def heatmap2d(df, column, palette='Inferno256', title='HeatMap', color_bar=True, handle=False):
    TOOLTIPS = [
    ("(x,y)", "(@x, @y)"),
    (f"{column}", f"@{column}"),
    ]
    
    p = figure(title=title, 
               tools=tools, 
               tooltips=TOOLTIPS,
               #x_range=(df['x'].min(), df['x'].max()), 
               #y_range=(df['y'].min(), df['y'].max()),
               match_aspect=True,
               sizing_mode='scale_both'               
              )
    
    mapper = LinearColorMapper(palette=palette, low=df[column].min(), high=df[column].max())

    p.rect(
        x="x",
        y="y",
        width=1,
        height=1,
        source=ColumnDataSource(df[['x','y',column]]),
        line_color=None,
        fill_color=transform(column, mapper),
        alpha=ALPHA,
        dilate=(True if ALPHA == 1.0 else False)
    )
    
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "5pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = 1.0

    if color_bar:
        # Add Legend Colorbar
        color_bar = ColorBar(
            color_mapper=mapper,
            location=(0, 0),
            ticker=BasicTicker(desired_num_ticks=int(len(mapper.palette)/5))
        )
        p.add_layout(color_bar, 'left')
    
    show(p)
    
    if handle:
        return p

In [13]:
def normalize_2d(matrix):
    return (matrix - np.min(matrix)) / (np.max(matrix) - np.min(matrix))

In [14]:
def path_loss_one_slope(d, ref_loss=REFERENCE_LOSS_1M, exponent=PATH_LOSS_EXP):
    return (ref_loss + 10*exponent*np.log10(d)) if d > 1 else ref_loss

In [15]:
#%%time
#Build lists for dataframe (this is way faster than appending data row-by-row on the dataframe)

x_list = []
y_list = []
dist_list = []
loss_list = []
rssi_list = []

for x in range(SIZE_X):
    for y in range(SIZE_Y):
        dist = distance2d(ORIGIN_X,ORIGIN_Y,x,y)
        loss = path_loss_one_slope(dist)
        rssi = TX_POWER - loss
        dist_list.append(dist)
        loss_list.append(loss)
        rssi_list.append(rssi)
        x_list.append(x)
        y_list.append(y)

#Build dataframe at once
df = pd.DataFrame({'x':x_list,'y':y_list,'dist':dist_list,'loss':loss_list,'rssi':rssi_list})

In [16]:
heatmap2d(df, 'rssi', title=f'RSSI Heatmap for AP on coordinates ({ORIGIN_X},{ORIGIN_Y})', palette=cc.kbc, color_bar=True)